In [1]:
1+1

2

In [2]:
import os
import logging
from dotenv import load_dotenv
from typing import TypedDict, List
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from langchain_openai import ChatOpenAI
from langchain.agents import Tool
from langgraph.prebuilt import create_react_agent
from langgraph.graph import StateGraph, END


# .env 파일에서 환경 변수를 로드합니다.
# OPENAI_API_KEY와 같은 민감한 정보를 코드에 직접 노출하지 않기 위함입니다.
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# 로깅 설정을 구성합니다.
# LangChain의 내부 동작을 자세히 보고 싶을 때 유용합니다.
# logging.basicConfig(level=logging.DEBUG) # 전체 로깅 활성화
logging.getLogger("langchain").setLevel(
    logging.DEBUG
)  # LangChain 관련 로그만 DEBUG 레벨로 설정

# -----------------------------------------------------------------
# 1. 도구(Tools) 정의
# -----------------------------------------------------------------
# 에이전트가 특정 작업을 수행하기 위해 호출할 수 있는 함수들입니다.


def add_two(x: str) -> str:
    """입력된 숫자에 2를 더하는 간단한 도구입니다."""
    try:
        # 입력 문자열을 부동소수점 숫자로 변환하여 2를 더하고, 다시 문자열로 반환합니다.
        return str(float(x) + 2)
    except Exception:
        # 숫자 변환에 실패하면 에러 메시지를 반환합니다.
        return "도구 실행 실패: 숫자만 입력해 주세요."


def population_lookup(city: str) -> str:
    """도시 이름을 받아 인구를 조회하는 도구입니다."""
    data = {"서울": "9,515,000명 (2023년 기준)", "부산": "3,343,000명 (2023년 기준)"}
    # .get() 메서드를 사용하면 키가 없을 때 None이나 지정된 기본값을 반환합니다.
    # return data.get(city.strip().lower(), f"{city}의 인구 정보를 찾을 수 없습니다.")

    # 여기서는 의도적으로 에러를 발생시켜 에이전트의 오류 처리 과정을 테스트합니다.
    # 에이전트는 이 에러를 보고, 도구 사용이 실패했음을 인지하게 됩니다.
    raise ValueError(f"{city}의 인구 정보를 찾을 수 없습니다.")


# -----------------------------------------------------------------
# 2. 상태(State) 정의
# -----------------------------------------------------------------
# LangGraph에서 각 노드를 거치며 공유되고 업데이트되는 데이터의 구조를 정의합니다.


class AgentState(TypedDict):
    """
    그래프의 상태를 나타냅니다. 상태는 노드 간에 전달되는 데이터입니다.
    'messages'는 사용자와 에이전트 간의 대화 기록을 담는 리스트입니다.
    """

    messages: List[BaseMessage]


# -----------------------------------------------------------------
# 3. 에이전트 및 그래프 구성
# -----------------------------------------------------------------

# 에이전트가 사용할 도구 리스트를 생성합니다.
tools = [
    Tool(name="AddTwo", func=add_two, description="숫자 입력 시 2를 더합니다."),
    Tool(
        name="PopulationLookup",
        func=population_lookup,
        description="대한민국 주요 도시의 인구를 조회합니다. (예: 서울, 부산)",
    ),
]

# LLM(거대 언어 모델)을 정의합니다. ReAct 에이전트의 두뇌 역할을 합니다.
# verbose=True: LLM의 내부 호출 과정을 로깅합니다. (LangGraph의 debug=True와는 별개)
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0, verbose=True)


# Fallback 로직: 에이전트가 실패했을 때 대신 실행될 순수 LLM 체인을 정의합니다.
# 참고: 현재 그래프 구조에서는 이 노드가 호출되지 않습니다.
#       조건부 엣지(Conditional Edge)를 추가해야 특정 상황에서 이 노드를 실행할 수 있습니다.
def create_llm_fallback():
    """순수 LLM으로만 응답을 생성하는 노드를 만듭니다."""

    def llm_only_response(state: AgentState) -> AgentState:
        # 마지막 사용자 메시지를 가져옵니다.
        last_message = state["messages"][-1]
        user_query = (
            last_message.content
            if hasattr(last_message, "content")
            else str(last_message)
        )

        # LLM에게 직접 질문하여 응답을 생성합니다.
        print("\n--- FALLBACK TO LLM ---")
        response = llm.invoke(f"다음 질문에 답해주세요: {user_query}")

        # 생성된 응답을 메시지 리스트에 추가하여 상태를 업데이트합니다.
        return {"messages": state["messages"] + [response]}

    return llm_only_response


# LangGraph를 사용하여 ReAct 에이전트를 생성합니다.
# model: 사용할 LLM
# tools: 에이전트가 사용할 도구
# debug=True: LangGraph의 실행 흐름을 단계별로 자세히 출력하여 디버깅에 용이합니다.
react_agent = create_react_agent(model=llm, tools=tools, debug=True)

# 워크플로우를 그래프로 정의합니다.
graph = StateGraph(AgentState)

# 그래프에 노드를 추가합니다. 각 노드는 작업 단위를 나타냅니다.
# "agent" 노드는 ReAct 로직을 실행합니다.
graph.add_node("agent", react_agent)
# "llm_fallback" 노드는 순수 LLM 응답을 생성합니다. (현재는 연결되지 않음)
graph.add_node("llm_fallback", create_llm_fallback())

# 그래프의 시작점을 "agent" 노드로 설정합니다.
graph.set_entry_point("agent")

# 노드 간의 연결(엣지)을 정의합니다.
# 현재는 "agent" 노드가 끝나면 무조건 END(종료)로 가도록 설정되어 있습니다.
# 에이전트 실패 시 "llm_fallback"으로 보내려면 조건부 엣지가 필요합니다.
graph.add_edge("agent", END)

# 정의된 그래프를 실행 가능한 애플리케이션으로 컴파일합니다.
app = graph.compile()


# -----------------------------------------------------------------
# 4. 에이전트 실행 및 테스트
# -----------------------------------------------------------------

# 에이전트가 무한 루프에 빠지는 것을 방지하기 위해 최대 반복 횟수를 설정합니다.
# ReAct는 (생각 -> 행동)이 한 사이클이므로, 2를 곱해줍니다.
max_iterations = 3
recursion_limit = 2 * max_iterations + 1

# response 변수를 미리 None으로 초기화합니다.
response = None

try:
    print(f"--- 에이전트 실행 시작 (최대 반복 {max_iterations}회) ---")
    # '대구'는 population_lookup 도구에 없는 도시이므로, 의도적으로 에러가 발생합니다.
    # 에이전트는 이 에러를 처리하려 시도하다가, 결국 최대 반복 횟수에 도달하게 됩니다.
    response = app.invoke(
        {"messages": [HumanMessage(content="대구의 인구는 몇 명이야?")]},
        {"recursion_limit": recursion_limit},
    )
except Exception as e:
    # 에이전트 실행 중 예외(예: 재귀 제한 도달)가 발생하면 여기에서 처리합니다.
    print(f"\n--- 에이전트 실행 중 오류 발생 ---")
    print(f"오류 타입: {type(e).__name__}")
    # 에이전트가 실패했을 때의 최종 상태를 임의로 구성해줄 수 있습니다.
    response = {
        "messages": [
            HumanMessage(content="대구의 인구는 몇 명이야?"),
            AIMessage(
                content="죄송합니다. 요청을 처리하는 데 실패했습니다. 내부적으로 오류가 발생했습니다."
            ),
        ]
    }


# 최종 결과 출력
print("\n--- 최종 대화 기록 ---")
if response and "messages" in response:
    for m in response["messages"]:
        # 각 메시지의 클래스 이름과 내용을 출력합니다.
        # getattr(m, "content", repr(m))는 content 속성이 없으면 객체의 표현을 출력합니다.
        print(f"[{m.__class__.__name__}]: {getattr(m, 'content', repr(m))}")
else:
    print("최종 응답을 가져오지 못했습니다.")

--- 에이전트 실행 시작 (최대 반복 3회) ---
[values] {'messages': [HumanMessage(content='대구의 인구는 몇 명이야?', additional_kwargs={}, response_metadata={}, id='f217d607-90fe-4ec6-b63d-ce25e7bce9a0')]}
[updates] {'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_jcdCmEkleDsUeQHuiPDw9qv8', 'function': {'arguments': '{"__arg1":"대구"}', 'name': 'PopulationLookup'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 103, 'total_tokens': 121, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BpbedW4vBeJx0ca7parWLANprMoZO', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--bccedec9-71e6-46ea-b34b-4a453bca86ff-0', tool_calls=[